<a href="https://colab.research.google.com/github/Aleksandar919/Principi-prezentacije1/blob/main/CukAleksandarIT74/2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UVOD

Age/Gender Recognition with Deep Learning Model // Prepoznavanje starosti/pola uz model dubokog učenja. Procena starosti lica na fotografiji može se postaviti kao duboki problem klasifikacije koristeći CNN praćen očekivanom preciziranjem softmax vrednosti (kao što se može uraditi sa modelom DeepEXpectation (DEX)).

Prikazaćemo kako se koristi unapred obučeni model dubokog učenja (WideResNet sa dva sloja klasifikacije dodata povrh njega, koji istovremeno procenjuje starost i pol pomoću jednog CNN-a) za prepoznavanje starosti i pola iz fotografije lica.

Za prepoznavanje starosti i pola koristićemo fotografije lica iz skupa podataka o poznatim licima. U izradi projekta koristi se dataset wiki_crop.

In [18]:
#Ucitavanje dataseta
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/file/d/19Ki9YhdluVkBFxYzZa4Sof1tzLW-QsNI/view?usp=sharing' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1oq1xgCGm47T5pEyjmbgxggQzzoEeENfT" -O images.zip && rm -rf /tmp/cookies.txt

--2021-09-25 19:35:02--  https://docs.google.com/uc?export=download&confirm=&id=1oq1xgCGm47T5pEyjmbgxggQzzoEeENfT
Resolving docs.google.com (docs.google.com)... 108.177.125.100, 108.177.125.138, 108.177.125.139, ...
Connecting to docs.google.com (docs.google.com)|108.177.125.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘images.zip’

images.zip              [ <=>                ]   3.21K  --.-KB/s    in 0s      

2021-09-25 19:35:03 (49.2 MB/s) - ‘images.zip’ saved [3291]



In [24]:
#Unzip images.zip
!unzip -q /My_Drive/content/images.zip 

unzip:  cannot find or open /My_Drive/content/images.zip, /My_Drive/content/images.zip.zip or /My_Drive/content/images.zip.ZIP.


Pre nego što se počne sa treniranjem i obradom podataka izvršeno je "pročišćavanje" podataka iz wiki-mat fajla. Postoji više problema u dataset-u. Slike nisu istih veličina, neke slike nemaju lica, godine su pogrešne itd.. Rešavanjem ovakvih problema dobija se dataset na kome je moguće vršiti Machine Learning. Na taj način dobija se dataset sa dva fajla age i gender. U svakom od fajlova nalaze se test i train podaci.

In [ ]:
# Import neophodnih modula
import numpy as np
from scipy.io import loadmat
import pandas as pd
import datetime as date
from dateutil.relativedelta import relativedelta

cols = ['age', 'gender', 'path', 'face_score1', 'face_score2']

wiki_mat = '/content/wiki_crop/wiki.mat'

wiki_data = loadmat(wiki_mat)

del wiki_mat

wiki = wiki_data['wiki']

wiki_photo_taken = wiki[0][0][1][0]
wiki_full_path = wiki[0][0][2][0]
wiki_gender = wiki[0][0][3][0]
wiki_face_score1 = wiki[0][0][6][0]
wiki_face_score2 = wiki[0][0][7][0]

wiki_path = []

for path in wiki_full_path:
    wiki_path.append('wiki_crop/' + path[0])

wiki_genders = []

for n in range(len(wiki_gender)):
    if wiki_gender[n] == 1:
        wiki_genders.append('male')
    else:
        wiki_genders.append('female')

wiki_dob = []

for file in wiki_path:
    wiki_dob.append(file.split('_')[2])

wiki_age = []

for i in range(len(wiki_dob)):
    try:
        d1 = date.datetime.strptime(wiki_dob[i][0:10], '%Y-%m-%d')
        d2 = date.datetime.strptime(str(wiki_photo_taken[i]), '%Y')
        rdelta = relativedelta(d2, d1)
        diff = rdelta.years
    except Exception as ex:
        print(ex)
        diff = -1
    wiki_age.append(diff)

final_wiki = np.vstack((wiki_age, wiki_genders, wiki_path, wiki_face_score1, wiki_face_score2)).T

final_wiki_df = pd.DataFrame(final_wiki)

final_wiki_df.columns = cols

meta = final_wiki_df

meta = meta[meta['face_score1'] != '-inf']
meta = meta[meta['face_score2'] == 'nan']

meta = meta.drop(['face_score1', 'face_score2'], axis=1)

meta = meta.sample(frac=1)

meta.to_csv('meta.csv', index=False)

In [ ]:
# Import neophodnih modula
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

# Učitavanje dataseta
meta = pd.read_csv('meta.csv')

# Brisanje kolone sa polom
meta = meta.drop(['gender'], axis=1)

# Filtriranje dataseta
meta = meta[meta['age'] >= 0]
meta = meta[meta['age'] <= 101]

# Konverzija vrednosti podataka u numpy array
meta = meta.values

# Podela dataseta na trening i testni skup
D_train, D_test = train_test_split(meta, test_size=0.2, random_state=42)

# Kreiranje svih neophodnih direktorijuma
for i in range(102):
    output_dir_train_male = 'dataset/age/train/' + str(i)
    output_dir_train_female = 'dataset/age/train/' + str(i)

    if not os.path.exists(output_dir_train_male):
        os.makedirs(output_dir_train_male)

    if not os.path.exists(output_dir_train_female):
        os.makedirs(output_dir_train_female)

    output_dir_test_male = 'dataset/age/test/' + str(i)
    output_dir_test_female = 'dataset/age/test/' + str(i)

    if not os.path.exists(output_dir_test_male):
        os.makedirs(output_dir_test_male)

    if not os.path.exists(output_dir_test_female):
        os.makedirs(output_dir_test_female)

# Kreiranje trening i testnog seta
counter = 0

for image in D_train:
    img = cv2.imread(image[1], 1)
    img = cv2.resize(img, (128,128))
    cv2.imwrite('dataset/age/train/' + str(image[0]) + '/' + str(counter) + '.jpg', img)
    print('--('+str(counter)+')Processing--')
    counter += 1

counter = 0

for image in D_test:
    img = cv2.imread(image[1], 1)
    img = cv2.resize(img, (128,128))
    cv2.imwrite('dataset/age/test/' + str(image[0]) +  '/' + str(counter) + '.jpg', img)
    print('--('+str(counter)+')Processing--')
    counter += 1

In [ ]:
# Import neophodnih modula
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

# Učitavanje podataka
meta = pd.read_csv('meta.csv')

# Brisanje kolone o godinama, s obzirom da mi više ne treba
meta = meta.drop(['age'], axis=1)

# Podela dataseta u trening i testni skup
D_train, D_test = train_test_split(meta, test_size=0.1, random_state=42)

# Dataset sadrži više slika lica osoba muškog pola nego ženskog --- ovo može da dovede do problema...
# Kako bi se rešio ovaj problem, selektuje se isti broj muških i ženskih lica u trening skupu
D_train_male = D_train[D_train['gender'] == 'male']
D_train_female = D_train[D_train['gender'] == 'female']

no_male = len(D_train_male)
no_female = len(D_train_female)

extra = D_train_male[no_female:]
D_train_male = D_train_male[0:no_female]

D_test = pd.concat((D_test, extra))

D_train = pd.concat((D_train_male, D_train_female))

# Shuffling dataseta
D_train = D_train.sample(frac=1)
D_test = D_test.sample(frac=1)

# Kreiranje direktorijuma
output_dir_train_male = 'dataset/gender/train/male'
output_dir_train_female = 'dataset/gender/train/female'

if not os.path.exists(output_dir_train_male):
    os.makedirs(output_dir_train_male)

if not os.path.exists(output_dir_train_female):
    os.makedirs(output_dir_train_female)

output_dir_test_male = 'dataset/gender/test/male'
output_dir_test_female = 'dataset/gender/test/female'

if not os.path.exists(output_dir_test_male):
    os.makedirs(output_dir_test_male)

if not os.path.exists(output_dir_test_female):
    os.makedirs(output_dir_test_female)

# Ubacivanje slika u trening i test skup
counter = 0

for image in D_train.values:
    img = cv2.imread(image[1], 1)
    img = cv2.resize(img, (128,128))
    if image[0] == 'male':
        cv2.imwrite('dataset/gender/train/male/' + str(counter) + '.jpg', img)
    else:
        cv2.imwrite('dataset/gender/train/female/' + str(counter) + '.jpg', img)
    print('--('+str(counter)+')Processing--')
    counter += 1

counter = 0

for image in D_test.values:
    img = cv2.imread(image[1], 1)
    img = cv2.resize(img, (128,128))
    if image[0] == 'male':
        cv2.imwrite('dataset/gender/test/male/' + str(counter) + '.jpg', img)
    else:
        cv2.imwrite('dataset/gender/test/female/' + str(counter) + '.jpg', img)
    print('--('+str(counter)+')Processing--')
    counter += 1

In [ ]:
!pip install Sequential

**Treniranje modela**

In [ ]:
# Import neophodnih modula
from keras.models import Sequential
from keras.layers import Dense, Flatten, MaxPooling2D, Conv2D, Dropout
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Kreiranje modela
model = Sequential()

In [ ]:
#Za ovaj model koristi se InceptionResNetV2
inception = InceptionResNetV2(include_top=False,
                          weights="imagenet", 
                          input_shape=(128,128,3),
                          pooling="max")

In [ ]:
#Dodavanje modela i konfigurisanje izlaza
model.add(inception)
model.add(Dense(units=102, activation="softmax"))

In [ ]:
#Kompajliranje modela
model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['accuracy'])

In [ ]:
#Prikaz podataka
print(model.summary())

In [ ]:
#Upotreba ImageDataGenerator klase 
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.3)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#Čitanje training seta
train_generator = train_datagen.flow_from_directory('dataset/age/train',
                                                    target_size=(128, 128),
                                                    batch_size=32,
                                                    class_mode='categorical')

In [ ]:
#Čitanje testnog seta
test_set = test_datagen.flow_from_directory('dataset/age/test',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='categorical')

In [ ]:
# Treniranje modela
model.fit_generator(train_generator,
                    steps_per_epoch=10,
                    epochs=2,
                    validation_data=test_set,
                    validation_steps=1500)

In [ ]:
#Čuvanje modela
model.save('weights/age_inception.h5')

Na identičan način odrađeno je i za GENDER.

In [ ]:
# Import neophodnih modula
from keras.models import Sequential
from keras.layers import Dense, Flatten, MaxPooling2D, Conv2D, Dropout
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Kreiranje modela
model = Sequential()

In [ ]:
#Za ovaj model koristi se InceptionResNetV2
inception = InceptionResNetV2(include_top=False,
                          weights="imagenet", 
                          input_shape=(128,128,3),
                          pooling="max")

In [ ]:
#Dodavanje modela i konfigurisanje izlaza
model.add(inception)
model.add(Dense(units=2, activation="softmax"))

In [ ]:
# Kompajliranje modela
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['accuracy'])

In [ ]:
# Prikaz podataka modela
print(model.summary())

In [ ]:
# Upotreba ImageDataGenerator klase 
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.3)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Čitanje training seta
train_generator = train_datagen.flow_from_directory('dataset/gender/train',
                                                    target_size=(128, 128),
                                                    batch_size=64,
                                                    class_mode='categorical')

In [ ]:
# Čitanje testnog seta
test_set = test_datagen.flow_from_directory('dataset/gender/test',
                                            target_size=(128, 128),
                                            batch_size=64,
                                            class_mode='categorical')

In [ ]:
# Treniranje modela
model.fit_generator(train_generator,
                    steps_per_epoch=10,
                    epochs=2,
                    validation_data=test_set,
                    validation_steps=800)

In [ ]:
# Čuvanje modela
model.save('weights/gender_inception.h5')

Model sa vežbi

In [ ]:
!wget -p /models https://github.com/yu4u/age-gender-estimation/releases/download/v0.5/weights.29-3.76_utk.hdf5

In [ ]:
#Učitavanje modela
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/file/d/1kvEeaHu335molambdynhxwx6cbp7D9ST/view?usp=sharing' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1oyfGIejVqiwE287uF6sU_LX5eHU2Pmgx" -O models.zip && rm -rf /tmp/cookies.txt

In [ ]:
#Unzipovanje modela sa vežbi
!unzip models.zip

In [ ]:
!pip install dlib

In [ ]:
#Import neopodnih modula
import sys
import numpy as np
from keras.models import Model
from keras import backend as K
import cv2
import dlib
from keras.models import model_from_json
from glob import glob
import matplotlib.pylab as plt

In [ ]:
depth = 16
k = width = 8
margin = 0.4
img_size = 64

In [ ]:
# Funkcija za iscrtavanje graničnih okvira sa oznakom godina/pola
def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1.2, thickness=3):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)

In [ ]:
# Frontalni detektor lica za otkrivanje i izdvajanje lica sa ulaznih fotografija
detector = dlib.get_frontal_face_detector()

plt.figure(figsize=(15,7))
plt.subplots_adjust(0,0,1,1,0.05,0.05)
j = 1

In [ ]:
"""
Za svako lice otkriveno na ulaznoj fotografiji pomoću dlib detektora, 
predviđa se starost i pol otkrivenog lica pomoću modela dubokog učenja pokretanjem prosleđivanja lica 
sa licem kao ulaznim podacima i izdvajaju predviđenu starost i pol.
"""
camera = cv2.VideoCapture(0)
haar = cv2.CascadeClassifier('cascades/haarcascade_frontalface_alt2.xml')

model = keras.models.load_model('/content/weights/gender_inception.h5')
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

for img_file in glob('/content/wiki_crop/00/1013900_1917-10-15_1960.jpg'): #['images/all.png']: 
    img = cv2.cvtColor(cv2.imread(img_file), cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(img)
    r = 640 / max(img_h, img_w)
    img = cv2.resize(img, (int(img_w * r), int(img_h * r)))

    # Detekcija lica korišćenjem dlib detektora
    detected = detector(img, 0) #0)

    faces = np.empty((len(detected), img_size, img_size, 3))
    
    if len(detected) > 0:
        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            faces[i, :, :, :] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))

        # Predikcija godina i pola detektovanih lica
        results = loaded_model.predict(faces)
        predicted_genders = results[0]
        ages = np.arange(0, 101).reshape(101, 1)
        predicted_ages = results[1].dot(ages).flatten()
        print(predicted_ages)

        # Iscrtavanje rezultata
        for i, d in enumerate(detected):
            label = "{}, {}".format(int(predicted_ages[i]), "F" if predicted_genders[i][0] > 0.5 else "M")
            draw_label(img, (d.left(), d.top()), label)

    plt.subplot(1,4,j), plt.imshow(img), plt.title(img_file.split('\\')[-1].split('.')[0], size=20), plt.axis('off')
    j += 1
plt.show()

In [ ]:
camera = cv2.VideoCapture(0)
haar = cv2.CascadeClassifier('cascades/haarcascade_frontalface_alt2.xml')

model = keras.models.load_model('/content/weights/gender_inception.h5')
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

for img_file in glob('/content/wiki_crop/00/10110600_1985-09-17_2012.jpg'): #['images/all.png']: 
    img = cv2.cvtColor(cv2.imread(img_file), cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(img)
    r = 640 / max(img_h, img_w)
    img = cv2.resize(img, (int(img_w * r), int(img_h * r)))

    # Detekcija lica korišćenjem dlib detektora
    detected = detector(img, 0) #0)

    faces = np.empty((len(detected), img_size, img_size, 3))
    
    if len(detected) > 0:
        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            faces[i, :, :, :] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))

        # Predikcija godina i pola detektovanih lica
        results = loaded_model.predict(faces)
        predicted_genders = results[0]
        ages = np.arange(0, 101).reshape(101, 1)
        predicted_ages = results[1].dot(ages).flatten()
        print(predicted_ages)

        # Iscrtavanje rezultata
        for i, d in enumerate(detected):
            label = "{}, {}".format(int(predicted_ages[i]), "F" if predicted_genders[i][0] > 0.5 else "M")
            draw_label(img, (d.left(), d.top()), label)

    plt.subplot(1,3,j), plt.imshow(img), plt.title(img_file.split('\\')[-1].split('.')[0], size=20), plt.axis('off')
    j += 1
plt.show()

Zaključak

Rad na ovom projektu doneo mi je još jedno dodatno iskustvo u oblasti Deep Learning-a. Iako se prvi put susrećem sa ovom temom, kao i rad u Colab-u (.ipynb) uspela sam uspešno da se izborim sa svim nastalim problemima i uspešno privedem projekat kraju. Daljim radom moguće je usavršavanje mreže, dodatni trening kao i učenje kako bi se što uspešnije detektovali odredjeni parametri.

